In [1]:
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=20)

import satpy
satpy.config.set({'cache_dir': "/home/ubuntu/data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from dask.diagnostics import Profiler, ResourceProfiler, visualize
from datetime import datetime
from satpy import Scene
from glob import glob
import numpy as np
import os

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from satpy.utils import debug_on
#debug_on()

In [ ]:
import dask
dask.config.set(scheduler='threads') 

curfile = "/home/ubuntu/data/him/HS_H09_20231029_0840_B07_FLDK_R20_S0110.DAT"
tod = "/home/ubuntu/data/out/"


bbox = (-2000000, -2770000, 790000, -940000)

comp = 'fire_radiative_power'

idirs = glob('/home/ubuntu/data/him/*')
idirs.sort()
idirs.reverse()
with dask.config.set({"array.chunk-size": "24MiB"}):
    for cdir in idirs:
        ifiles_l15 = glob(cdir+'/*.DAT')
        if len(ifiles_l15) < 5:
            continue
        curf = ifiles_l15[0]
        pos = curf.find('HS_H')
        dtstr = curf[pos+7:pos+7+13]
        outf = f'{tod}/{comp}_{dtstr}00.tif'
        print(outf)
        
        st = datetime.utcnow()
        if not os.path.isfile(outf):
            scn = Scene(ifiles_l15, reader='ahi_hsd')
            scn.load([comp], generate=False)
            #scn.load(['B07'], calibration='radiance')
            #scn.load([comp], generate=False)
           # scn2 = scn.crop(xy_bbox = bbox)
            scn3 = scn.resample(scn.coarsest_area(), resampler='native')
           # scn3.save_datasets(filename=outf,  fill_value=0, writer='simple_image')
            scn3.save_datasets(base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)

            en = datetime.utcnow()
            print(dtstr, (en-st).total_seconds())

/home/ubuntu/data/out//fire_radiative_power_20240415_235000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_234000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_233000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_232000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_231000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_230000.tif
/home/ubuntu/data/out//fire_radiative_power_20240415_225000.tif
